In [1]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
 

ModuleNotFoundError: No module named 'numpy'

In [ ]:
data = pd.read_csv("beer-servings.csv")
data.duplicated().sum()

In [ ]:
data.isna().sum()

splitting numerical and categorical columns

In [ ]:
num_df = data.select_dtypes(include="number")
cat_df = data.select_dtypes(include="object")

print(num_df)
print(cat_df)

Histogram of the numerical data

In [ ]:
num_cols = num_df.columns.tolist()
print(num_cols)
for col in num_cols:
    plt.hist(num_df[col].dropna())
    plt.xlabel(col)
    plt.ylabel("count")
    plt.show()
    

In [ ]:
for col in num_cols:
    num_df[col] = num_df[col].fillna(num_df[col].median())
    
print(num_df.isna().sum())

In [ ]:
plt.boxplot(num_df)
plt.show()

In [ ]:
def clip_outliers(df, column_name):
    q1 = df[column_name].quantile(0.25)
    q3 = df[column_name].quantile(0.75)
    iqr = q3 - q1
    upper_bound = q3 + 1.5 *iqr
    lower_bound = q1 - 1.5 *iqr
    df[column_name] = df[column_name].clip(upper=upper_bound)
    df[column_name] = df[column_name].clip(lower=lower_bound)
    return df[column_name]

for col in num_cols:
    num_df[col] = clip_outliers(num_df, col)
    
plt.boxplot(num_df)
plt.show()

Min max scaling and standard scaling


In [ ]:
num_df.head(2)

In [ ]:
# Min max scaling and standard scaling without library
for col in num_cols[:-1]:
    num_df[col] = (num_df[col]- num_df[col.min()])/(num_df[col].max()- num_df[col].min())
    
for col in num_cols[:-1]:
    num_df[col] = (num_df[col]- num_df[col].mean())/num_df[col].std()    

In [ ]:
# mix max and standard scaling with library

from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()
num_df = min_max_scaler.fit_transform(num_df)
print(num_df)

In [ ]:
# mix max and standard scaling with library

from sklearn.preprocessing import StandardScaler
std_scaler = StandardScaler()
num_df = std_scaler.fit_transform(num_df)
print(num_df)

In [ ]:
# saving the min max and standard as files

import pickle
with open("min_max_scaler.pkl", "wb") as f:
   pickle.dump(min_max_scaler, f)

In [ ]:
with open("min_max_scaler.pkl", "rb") as f:
    min_max_scaler = pickle.load(f)

print("Min:", min_max_scaler.min_)
print("Scale:", min_max_scaler.scale_)
print("Feature range:", min_max_scaler.feature_range)

In [ ]:
test_input = np.array([[200, 100, 300, 150]])
test_output = min_max_scaler.transform(test_input)
print(test_output)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
data = {'color': ['yellow', 'green', 'blue', 'yellow', 'green']}
df = pd.DataFrame(data)
encoder = OneHotEncoder(sparse_output=False)
encoded_data = encoder.fit_transform(df[['color']])
encoded_df = pd.DataFrame(encoded_data, columns=['blue', 'green', 'yellow'])
final_df = pd.concat([df, encoded_df] , axis=1)
print(final_df)

In [ ]:
cat_cols = cat_df.columns.tolist()
for col in cat_cols:
    encoder = LabelEncoder()
    cat_df[col] = encoder.fit_transform(cat_df[col])
print(cat_df.head())


In [ ]:
# Assuming you want to use the original dataframe from 'data' (beer-servings.csv)
# and not the example 'final_df' from the color encoding example

# First, let's reconstruct a DataFrame from the scaled numpy array for easier column access
num_df_scaled = pd.DataFrame(num_df, columns=num_cols)

y = num_df_scaled["total_litres_of_pure_alcohol"]
X = num_df_scaled.drop("total_litres_of_pure_alcohol", axis=1)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train